In [30]:
import pandas as pd
import numpy as np
import seaborn as sns
import sqlalchemy as sqla
import csv
import matplotlib.pyplot as plt

DB_USER = "postgres"
DB_PASSWORD = "a"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "postgres"

connection_string = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Créer un moteur SQLAlchemy
engine = sqla.create_engine(connection_string)

try:
    with engine.connect() as connection:
        print("Connected to PostgreSQL successfully!")
except Exception as e:
    print(f"Error: {e}")

connection = engine.connect()

Connected to PostgreSQL successfully!


In [20]:
query = """
SELECT * FROM stations;
"""

stations = pd.read_sql(sqla.text(query), connection)

data_province = [
    (0,"Bruxelles", 4.367414882050033, 50.83642580516115),
    (1,"Antwerpen", 4.721043779498864, 51.231900712244745),
    (2,"Limburg", 5.4357209224443475, 50.98831159244354),
    (3,"Oost-Vlaanderen", 3.8188605810442007, 51.03629347952689),
    (4,"Vlaams Brabant", 4.59072072950776, 50.87303943005528),
    (5,"West-Vlaanderen", 3.0620446668608117, 51.01019740435232),
    (6,"Brabant Wallon", 4.589736973914637, 50.66605965845173),
    (7,"Hainaut", 3.9681364929631004, 50.46387637972707),
    (8,"Liège", 5.7372263688290746, 50.518876085615986),
    (9,"Luxembourg", 5.516735682549194, 49.95849015160492),
    (10,"Namur", 4.862346452029854, 50.252686235186815),
]

province = pd.DataFrame(data_province, columns=["province_id", "province_name", "province_long", "province_lat"])

def haversine(lon1, lat1, lon2, lat2):
    R = 6371  
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

def find_nearest_province(station_row):
    distances = haversine(
        station_row['longitude'], station_row['latitude'],
        province['province_long'].values, province['province_lat'].values
    )
    nearest_idx = distances.argmin()
    return province.iloc[nearest_idx]

# Apply the function
nearest_provinces = stations.apply(find_nearest_province, axis=1)

# Concatenate station info with nearest province
stations_with_province = pd.concat([stations.reset_index(drop=True), nearest_provinces.reset_index(drop=True)], axis=1)

road = [159,151,149,150,156,157,154,245,773,132,310,365,422,300,628,614,591,
 540,118,178,235,281,480,108,642,625,623,557,558,384,199,559,645,123,195,73,
 261,560,187,438,172,342,341,80,503,629,267,313,608,339,452,564,493,
 317,704,757,70]

filtered_stations = stations_with_province[stations_with_province['id'].isin(road)]
filtered_stations = filtered_stations.drop(columns=['province_long', 'province_lat','region', 'latitude', 'longitude','geom', 'name', 'province_name'])
filtered_stations['id'] = pd.Categorical(filtered_stations['id'], categories=road, ordered=True)
filtered_stations = filtered_stations.sort_values('id').reset_index(drop=True)

print(filtered_stations.head())

    id  province_id
0  159            0
1  151            0
2  149            0
3  150            0
4  156            0


In [12]:
query = """
SELECT 
    td.id,
    td.departure_date,
    td.train_number,
    td.relation,
    td.train_service,
    td.ptcar_number,
    td.line_number_departure,
    td.real_time_arrival,
    td.real_time_departure,
    td.planned_time_arrival,
    td.planned_time_departure,
    td.delay_arrival,
    td.delay_departure,
    ptcar.id AS ptcar_id,     
    ptcar.name AS name_travel,    
    td.line_number_arrival,
    dep.name AS departure_station_name,
    arr.name AS arrival_station_name,
    ptcar.longitude AS ptcar_longitude,  
    ptcar.latitude AS ptcar_latitude  
FROM train_data td
JOIN STATIONS ptcar ON td.ptcar_name = ptcar.id
JOIN STATIONS dep ON td.station_departure = dep.id
JOIN STATIONS arr ON td.station_arrival = arr.id
WHERE td.station_arrival = '487'  
  AND td.station_departure = '159'  
ORDER BY td.id;
"""

line = pd.read_sql(sqla.text(query), connection)
line =line.drop(columns=['train_service', 'relation'])

road = [159,151,149,150,156,157,154,245,773,132,310,365,422,300,628,614,591,
 540,118,178,235,281,480,108,642,625,623,557,558,384,199,559,645,123,195,73,
 261,560,187,438,172,342,341,80,503,629,267,313,608,339,452,564,493,
 317,704,757,70]

print("Road:", len(road))

tempo = []
clean = []
road_index = 0

for i, row in line.iterrows():
    ptcar_id = row['ptcar_id']
    if ptcar_id == road[road_index]:
        tempo.append(row)
        road_index += 1
        if len(tempo) == len(road):
            clean.extend(tempo)
            tempo = []
            road_index = 0
    else:
        tempo = []
        road_index = 0

clean_df = pd.DataFrame(clean)

query = """
    SELECT * FROM TYPE_DAY;
"""

type_day = pd.read_sql(sqla.text(query), connection)
clean_df= clean_df.drop(columns=['departure_station_name','arrival_station_name','id', 'name_travel', 'ptcar_number','ptcar_id'])
merged_df = pd.merge(clean_df, type_day, left_on='departure_date', right_on='date', how='left')
merged_df.loc[merged_df.index[::57], "delay_arrival"] = 0

merged_df.loc[merged_df.index[::57], "real_time_arrival"] = merged_df.loc[merged_df.index[::57], "real_time_departure"]
merged_df.loc[merged_df.index[::57], "planned_time_arrival"] = merged_df.loc[merged_df.index[::57], "planned_time_departure"]
merged_df.loc[merged_df.index[::57], "line_number_arrival"] = merged_df.loc[merged_df.index[::57], "line_number_departure"]


merged_df.loc[merged_df.index[56::57], "line_number_departure"] = 162
merged_df.loc[merged_df.index[56::57], "line_number_arrival"] = 162

index = merged_df.index[56::57]
mask = merged_df.loc[index, "real_time_departure"] == None
merged_df.loc[index, "real_time_departure"] = merged_df.loc[index, "real_time_arrival"]

mask = merged_df.loc[index, "planned_time_departure"] == None
merged_df.loc[index, "planned_time_departure"] = merged_df.loc[index, "planned_time_arrival"]

mask = merged_df.loc[index, "delay_departure"] == None
merged_df.loc[index, "delay_departure"] = merged_df.loc[index, "delay_arrival"]


Road: 57


In [16]:
print(filtered_stations.shape)

(57, 2)


In [22]:
station_ids = filtered_stations['province_id'].values
total_rows = 1558950
repeated_ids = np.tile(station_ids, total_rows // len(station_ids) + 1)[:total_rows]

repeated_ids_df = pd.DataFrame({'repeated_station_id': repeated_ids})

print("Station IDs:", station_ids)
print(repeated_ids_df.shape)

Station IDs: [ 0  0  0  0  0  0  0  0  0  0  0  0  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6 10 10 10 10 10 10  6 10 10 10 10 10 10 10 10 10 10  9  9  9  9
  9  9  9  9  9  9  9  9  9]
(1558950, 1)


In [15]:
print(merged_df.shape)

(1558950, 16)


In [24]:
merged_df = pd.concat([merged_df, repeated_ids_df], axis=1)
print(merged_df.shape)

(1558950, 17)


In [ ]:
query = """
SELECT 
    w.date,
    EXTRACT(HOUR FROM w.hour) AS hour_of_day,
    w.temperature AS temperature,
    w.dewpoint AS dewpoint,
    w.relative_humidity AS relative_humidity,
    w.precipitation AS precipitation,
    w.snowfall AS snowfall,
    w.wind_direction AS wind_direction,
    w.wind_speed AS wind_speed,
    w.pressure AS pressure,
    w.province AS province_id
FROM WEATHER w
GROUP BY 
    w.date,
    EXTRACT(HOUR FROM w.hour),
    w.temperature,
    w.dewpoint,
    w.relative_humidity,
    w.precipitation,
    w.snowfall,
    w.wind_direction,
    w.wind_speed,
    w.pressure,
    w.province
ORDER BY w.date, hour_of_day;

"""

weather = pd.read_sql(sqla.text(query), connection)

print(weather.head())


         date  hour_of_day  temperature  dewpoint  relative_humidity  \
0  2014-01-01          0.0          2.6       2.2               97.0   
1  2014-01-01          0.0          2.8      -0.3               91.0   
2  2014-01-01          0.0          4.9       4.3               96.0   
3  2014-01-01          0.0          5.5       4.6               94.0   
4  2014-01-01          0.0          5.5       4.6               94.0   

   precipitation  snowfall  wind_direction  wind_speed  pressure  province_id  
0            0.0       0.0           210.0        10.8    1011.2            9  
1            0.5       0.0           190.0        21.6    1011.3            8  
2            0.0       0.0           220.0        18.0    1010.2           10  
3            0.0       0.0           210.0        18.0    1009.2            7  
4            0.0       0.0           210.0        21.6    1009.2            6  


In [32]:
merged_df['time'] = pd.to_datetime(merged_df['planned_time_arrival'], format='%H:%M:%S')
merged_df['hour'] = merged_df['time'].dt.hour

In [34]:
print(merged_df.head)

<bound method NDFrame.head of         departure_date  train_number line_number_departure real_time_arrival  \
0           2017-06-11          2119                   0/1          19:35:25   
1           2017-06-11          2119                   0/1          19:37:30   
2           2017-06-11          2119                   0/1          19:38:48   
3           2017-06-11          2119                   0/1          19:41:36   
4           2017-06-11          2119                   36N          19:43:27   
...                ...           ...                   ...               ...   
1558945     2024-04-12          2114                   162          17:31:51   
1558946     2024-04-12          2114                   162          17:37:11   
1558947     2024-04-12          2114                   162          17:41:24   
1558948     2024-04-12          2114                   162          17:42:11   
1558949     2024-04-12          2114                   162          17:44:41   

        r

In [35]:
final_df = pd.merge(merged_df, weather, left_on=['departure_date', 'hour','repeated_station_id'], right_on=['date', 'hour_of_day','province_id'], how='left')
final_df = final_df.drop(columns=['date_x', 'date_y', 'hour_of_day','time','hour'])

In [36]:
print(final_df.head())

  departure_date  train_number line_number_departure real_time_arrival  \
0     2017-06-11          2119                   0/1          19:35:25   
1     2017-06-11          2119                   0/1          19:37:30   
2     2017-06-11          2119                   0/1          19:38:48   
3     2017-06-11          2119                   0/1          19:41:36   
4     2017-06-11          2119                   36N          19:43:27   

  real_time_departure planned_time_arrival planned_time_departure  \
0            19:35:25             19:33:00               19:33:00   
1            19:37:30             19:35:00               19:35:00   
2            19:39:51             19:36:00               19:37:00   
3            19:41:36             19:39:00               19:39:00   
4            19:45:12             19:41:00               19:43:00   

   delay_arrival  delay_departure line_number_arrival  ...  \
0            0.0            145.0                 0/1  ...   
1          150.0

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import sqlalchemy as sqla
import csv
import matplotlib.pyplot as plt

DB_USER = "postgres"
DB_PASSWORD = "a"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "postgres"

connection_string = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Créer un moteur SQLAlchemy
engine = sqla.create_engine(connection_string)

try:
    with engine.connect() as connection:
        print("Connected to PostgreSQL successfully!")
except Exception as e:
    print(f"Error: {e}")

connection = engine.connect()

query = """
SELECT * FROM stations;
"""

stations = pd.read_sql(sqla.text(query), connection)

data_province = [
    (0,"Bruxelles", 4.367414882050033, 50.83642580516115),
    (1,"Antwerpen", 4.721043779498864, 51.231900712244745),
    (2,"Limburg", 5.4357209224443475, 50.98831159244354),
    (3,"Oost-Vlaanderen", 3.8188605810442007, 51.03629347952689),
    (4,"Vlaams Brabant", 4.59072072950776, 50.87303943005528),
    (5,"West-Vlaanderen", 3.0620446668608117, 51.01019740435232),
    (6,"Brabant Wallon", 4.589736973914637, 50.66605965845173),
    (7,"Hainaut", 3.9681364929631004, 50.46387637972707),
    (8,"Liège", 5.7372263688290746, 50.518876085615986),
    (9,"Luxembourg", 5.516735682549194, 49.95849015160492),
    (10,"Namur", 4.862346452029854, 50.252686235186815),
]

province = pd.DataFrame(data_province, columns=["province_id", "province_name", "province_long", "province_lat"])

def haversine(lon1, lat1, lon2, lat2):
    R = 6371  
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

def find_nearest_province(station_row):
    distances = haversine(
        station_row['longitude'], station_row['latitude'],
        province['province_long'].values, province['province_lat'].values
    )
    nearest_idx = distances.argmin()
    return province.iloc[nearest_idx]

# Apply the function
nearest_provinces = stations.apply(find_nearest_province, axis=1)

# Concatenate station info with nearest province
stations_with_province = pd.concat([stations.reset_index(drop=True), nearest_provinces.reset_index(drop=True)], axis=1)

road = [159,151,149,150,156,157,154,245,773,132,310,365,422,300,628,614,591,
 540,118,178,235,281,480,108,642,625,623,557,558,384,199,559,645,123,195,73,
 261,560,187,438,172,342,341,80,503,629,267,313,608,339,452,564,493,
 317,704,757,70]

filtered_stations = stations_with_province[stations_with_province['id'].isin(road)]
filtered_stations = filtered_stations.drop(columns=['province_long', 'province_lat','region', 'latitude', 'longitude','geom', 'name', 'province_name'])
filtered_stations['id'] = pd.Categorical(filtered_stations['id'], categories=road, ordered=True)
filtered_stations = filtered_stations.sort_values('id').reset_index(drop=True)

query = """
SELECT 
    td.id,
    td.departure_date,
    td.train_number,
    td.relation,
    td.train_service,
    td.ptcar_number,
    td.line_number_departure,
    td.real_time_arrival,
    td.real_time_departure,
    td.planned_time_arrival,
    td.planned_time_departure,
    td.delay_arrival,
    td.delay_departure,
    ptcar.id AS ptcar_id,     
    ptcar.name AS name_travel,    
    td.line_number_arrival,
    dep.name AS departure_station_name,
    arr.name AS arrival_station_name,
    ptcar.longitude AS ptcar_longitude,  
    ptcar.latitude AS ptcar_latitude  
FROM train_data td
JOIN STATIONS ptcar ON td.ptcar_name = ptcar.id
JOIN STATIONS dep ON td.station_departure = dep.id
JOIN STATIONS arr ON td.station_arrival = arr.id
WHERE td.station_arrival = '487'  
  AND td.station_departure = '159'  
ORDER BY td.id;
"""

line = pd.read_sql(sqla.text(query), connection)
line =line.drop(columns=['train_service', 'relation'])
print("Road:", len(road))

tempo = []
clean = []
road_index = 0

for i, row in line.iterrows():
    ptcar_id = row['ptcar_id']
    if ptcar_id == road[road_index]:
        tempo.append(row)
        road_index += 1
        if len(tempo) == len(road):
            clean.extend(tempo)
            tempo = []
            road_index = 0
    else:
        tempo = []
        road_index = 0

clean_df = pd.DataFrame(clean)

query = """
    SELECT * FROM TYPE_DAY;
"""


type_day = pd.read_sql(sqla.text(query), connection)
clean_df= clean_df.drop(columns=['departure_station_name','arrival_station_name','id', 'name_travel', 'ptcar_number','ptcar_id'])
merged_df = pd.merge(clean_df, type_day, left_on='departure_date', right_on='date', how='left')
merged_df.loc[merged_df.index[::57], "delay_arrival"] = 0

merged_df.loc[merged_df.index[::57], "real_time_arrival"] = merged_df.loc[merged_df.index[::57], "real_time_departure"]
merged_df.loc[merged_df.index[::57], "planned_time_arrival"] = merged_df.loc[merged_df.index[::57], "planned_time_departure"]
merged_df.loc[merged_df.index[::57], "line_number_arrival"] = merged_df.loc[merged_df.index[::57], "line_number_departure"]


merged_df.loc[merged_df.index[56::57], "line_number_departure"] = 162
merged_df.loc[merged_df.index[56::57], "line_number_arrival"] = 162

index = merged_df.index[56::57]
mask = merged_df.loc[index, "real_time_departure"] == None
merged_df.loc[index, "real_time_departure"] = merged_df.loc[index, "real_time_arrival"]

mask = merged_df.loc[index, "planned_time_departure"] == None
merged_df.loc[index, "planned_time_departure"] = merged_df.loc[index, "planned_time_arrival"]

mask = merged_df.loc[index, "delay_departure"] == None
merged_df.loc[index, "delay_departure"] = merged_df.loc[index, "delay_arrival"]

station_ids = filtered_stations['province_id'].values
total_rows = 1558950
repeated_ids = np.tile(station_ids, total_rows // len(station_ids) + 1)[:total_rows]

repeated_ids_df = pd.DataFrame({'repeated_station_id': repeated_ids})
merged_df = pd.concat([merged_df, repeated_ids_df], axis=1)
query = """
SELECT 
    w.date,
    EXTRACT(HOUR FROM w.hour) AS hour_of_day,
    w.temperature AS temperature,
    w.dewpoint AS dewpoint,
    w.relative_humidity AS relative_humidity,
    w.precipitation AS precipitation,
    w.snowfall AS snowfall,
    w.wind_direction AS wind_direction,
    w.wind_speed AS wind_speed,
    w.pressure AS pressure,
    w.province AS province_id
FROM WEATHER w
GROUP BY 
    w.date,
    EXTRACT(HOUR FROM w.hour),
    w.temperature,
    w.dewpoint,
    w.relative_humidity,
    w.precipitation,
    w.snowfall,
    w.wind_direction,
    w.wind_speed,
    w.pressure,
    w.province
ORDER BY w.date, hour_of_day;

"""

weather = pd.read_sql(sqla.text(query), connection)

merged_df['time'] = pd.to_datetime(merged_df['planned_time_arrival'], format='%H:%M:%S')
merged_df['hour'] = merged_df['time'].dt.hour

final_df = pd.merge(merged_df, weather, left_on=['departure_date', 'hour','repeated_station_id'], right_on=['date', 'hour_of_day','province_id'], how='left')
final_df = final_df.drop(columns=['date_x', 'date_y', 'hour_of_day','time','hour','repeated_station_id'])
print("Middle of processing")
def parse_time_to_seconds(val):
    if pd.isna(val):
        return 0
    try:
        if isinstance(val, (int, float)):
            return int(val)
        if str(val).isdigit():
            return int(val)
        t = pd.to_datetime(val, format='%H:%M:%S', errors='coerce')
        if pd.isna(t):
            return 0
        return t.hour * 3600 + t.minute * 60 + t.second
    except Exception:
        return 0


final_df['departure_date'] = pd.to_datetime(final_df['departure_date'], format='%Y-%m-%d')

final_df['departure_year'] = final_df['departure_date'].dt.year
final_df['departure_month'] = final_df['departure_date'].dt.month
final_df['departure_day'] = final_df['departure_date'].dt.day
final_df['departure_weekday'] = final_df['departure_date'].dt.weekday

final_df['real_time_arrival'] = final_df['real_time_arrival'].apply(parse_time_to_seconds)
final_df['planned_time_arrival'] = final_df['planned_time_arrival'].apply(parse_time_to_seconds)
final_df['planned_time_departure'] = final_df['planned_time_departure'].apply(parse_time_to_seconds)
final_df['real_time_departure'] = final_df['real_time_departure'].apply(parse_time_to_seconds)

final_df = final_df.drop(columns=['departure_date', 'line_number_departure', 'line_number_arrival' ,'train_number'])
final_df['delay_arrival'] = final_df['delay_arrival'].apply(parse_time_to_seconds)

trajet_valid = []

for i in range(0, len(final_df), 57):
    trajet = final_df.iloc[i:i+57]
    if trajet.shape[0] < 57:
        print("Not enough data for this trajet, skipping")
        continue

    if trajet.isnull().values.any():
        continue

    if trajet['delay_arrival'].abs().max() > 36000:
        print("Delay arrival exceeds 10000 seconds, skipping")
        continue

    trajet_valid.append(trajet)

trajet_valid_df = pd.concat(trajet_valid, ignore_index=True)
trajet_valid_df["stop_index"] = trajet_valid_df.index % 57


Connected to PostgreSQL successfully!
Road: 57
Skipping trajet starting at index 18069: contains NaN
Skipping trajet starting at index 18240: contains NaN
Skipping trajet starting at index 18468: contains NaN
Skipping trajet starting at index 18525: contains NaN
Skipping trajet starting at index 18753: contains NaN
Skipping trajet starting at index 18810: contains NaN
Skipping trajet starting at index 18924: contains NaN
Skipping trajet starting at index 19038: contains NaN
Skipping trajet starting at index 19095: contains NaN
Skipping trajet starting at index 45771: contains NaN
Skipping trajet starting at index 45942: contains NaN
Skipping trajet starting at index 46113: contains NaN
Skipping trajet starting at index 46284: contains NaN
Skipping trajet starting at index 63099: contains NaN
Skipping trajet starting at index 63783: contains NaN
Skipping trajet starting at index 66462: contains NaN
Skipping trajet starting at index 66804: contains NaN
Skipping trajet starting at index 6

In [39]:
print(trajet_valid_df.shape)

(1555131, 25)
